In [15]:
import os
import fastf1 as ff1
import fastf1.plotting
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import requests
import dotenv

warnings.filterwarnings('ignore')

# Create cache directory if it doesn't exist
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache(cache_dir)

In [16]:
schedule = ff1.get_event_schedule(2025)
schedule[schedule['Country'] == 'Australia']

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Australia,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16,Australian Grand Prix,conventional,Practice 1,2025-03-14 12:30:00+11:00,2025-03-14 01:30:00,...,Practice 3,2025-03-15 12:30:00+11:00,2025-03-15 01:30:00,Qualifying,2025-03-15 16:00:00+11:00,2025-03-15 05:00:00,Race,2025-03-16 15:00:00+11:00,2025-03-16 04:00:00,True


In [ ]:
def get_event_data(year, event_name):
    '''
    Get each session data for a given event and year, including weather conditions.

    Arguments:
    year -- int, the year of the event
    event_name -- str, the name of the event

    Returns:
    pd.DataFrame -- DataFrame containing driver data and weather conditions for each session
    '''
    session_types = ['FP1', 'FP2', 'FP3', 'Q', 'R']
    final_results = {}

    for session_type in session_types:
        try:
            session = ff1.get_session(year, event_name, session_type)
            session.load()

            # Add weather condition for each session
            weather = session.weather_data
            weather_condition = {}
            if weather is not None and not weather.empty:
                weather_condition = {
                    f'{session_type}_air_temp': weather['AirTemp'].mean(),
                    f'{session_type}_humidity': weather['Humidity'].mean(),
                    f'{session_type}_rainfall': weather['Rainfall'].mean(),
                    f'{session_type}_pressure': weather['Pressure'].mean(),
                    f'{session_type}_wind_speed': weather['WindSpeed'].mean(),
                    f'{session_type}_wind_direction': weather['WindDirection'].mean(),
                    f'{session_type}_track_temp': weather['TrackTemp'].mean()
                }

            # Choose unique drivers in the session
            drivers = session.laps['Driver'].unique()
            for driver in drivers:

                # Initialize driver entry if not already present
                if driver not in final_results:
                    
                    # Basic driver info
                    final_results[driver] = {
                        'driver_number': session.laps.loc[session.laps['Driver'] == driver, 'DriverNumber'].iloc[0],
                        'driver': driver,
                        'year': year,
                        'event': event_name,
                        'team': session.laps.loc[session.laps['Driver'] == driver, 'Team'].iloc[0],
                    }
                driver_laps = session.laps.pick_driver(driver)
                
                # For each driver laps, get average sector times and total time
                if not driver_laps.empty:
                    sector_times = driver_laps[['Sector1Time', 'Sector2Time', 'Sector3Time']].mean().dropna()             
                    total_times = sector_times.sum().total_seconds() if not sector_times.isna().any() else None
                    final_results[driver] = {
                        **final_results[driver],
                        **weather_condition,
                        f'{session_type}_total_time': total_times
                    }
            
            # In race, get final classified position
            if session_type == 'R':
                result_positions = session.results[['ClassifiedPosition', 'Abbreviation']]
                result_positions_df = pd.DataFrame(result_positions).set_index('Abbreviation')
                for driver in drivers:
                    if driver in result_positions_df.index:
                        final_results[driver]['race_position'] = result_positions_df.loc[driver, 'ClassifiedPosition']
   
        except Exception as e:
            print(f"Could not load session {session_type} for {event_name} {year}: {e}")
            continue

    df = pd.DataFrame(final_results).T

    return df

In [18]:
australian_gp_2025_data = get_event_data(2025, 'Australian Grand Prix')
australian_gp_2025_data

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.6.1]
req            INFO 	Using cached data for session_info


req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_s

,driver_number,driver,year,event,team,FP1_air_temp,FP1_humidity,FP1_rainfall,FP1_pressure,FP1_wind_speed,...,FP3_total_time,Q_air_temp,Q_humidity,Q_rainfall,Q_pressure,Q_wind_speed,Q_wind_direction,Q_track_temp,Q_total_time,race_position
VER,1,VER,2025,Australian Grand Prix,Red Bull Racing,24.403704,47.876543,0.0,1018.146914,0.862963,...,109.207585,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,94.80847,2
NOR,4,NOR,2025,Australian Grand Prix,McLaren,24.403704,47.876543,0.0,1018.146914,0.862963,...,108.397392,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,98.074042,1
BOR,5,BOR,2025,Australian Grand Prix,Kick Sauber,24.403704,47.876543,0.0,1018.146914,0.862963,...,112.398633,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,103.647504,R
HAD,6,HAD,2025,Australian Grand Prix,Racing Bulls,24.403704,47.876543,0.0,1018.146914,0.862963,...,108.71865,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,100.048166,R
DOO,7,DOO,2025,Australian Grand Prix,Alpine,24.403704,47.876543,0.0,1018.146914,0.862963,...,116.110055,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,100.535957,R
GAS,10,GAS,2025,Australian Grand Prix,Alpine,24.403704,47.876543,0.0,1018.146914,0.862963,...,113.662822,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,98.668016,11
ANT,12,ANT,2025,Australian Grand Prix,Mercedes,24.403704,47.876543,0.0,1018.146914,0.862963,...,103.367126,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,96.97973,4
ALO,14,ALO,2025,Australian Grand Prix,Aston Martin,24.403704,47.876543,0.0,1018.146914,0.862963,...,102.128944,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,98.904134,R
LEC,16,LEC,2025,Australian Grand Prix,Ferrari,24.403704,47.876543,0.0,1018.146914,0.862963,...,116.384202,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,97.824023,8
STR,18,STR,2025,Australian Grand Prix,Aston Martin,24.403704,47.876543,0.0,1018.146914,0.862963,...,102.945878,30.336,46.146667,0.0,1009.376,0.772,159.6,40.501333,95.810499,6


In [19]:
australian_gp_2025_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, VER to BEA
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   driver_number       20 non-null     object
 1   driver              20 non-null     object
 2   year                20 non-null     object
 3   event               20 non-null     object
 4   team                20 non-null     object
 5   FP1_air_temp        20 non-null     object
 6   FP1_humidity        20 non-null     object
 7   FP1_rainfall        20 non-null     object
 8   FP1_pressure        20 non-null     object
 9   FP1_wind_speed      20 non-null     object
 10  FP1_wind_direction  20 non-null     object
 11  FP1_track_temp      20 non-null     object
 12  FP1_total_time      20 non-null     object
 13  FP2_air_temp        19 non-null     object
 14  FP2_humidity        19 non-null     object
 15  FP2_rainfall        19 non-null     object
 16  FP2_pressure        19 non-nul